## Initiate open_clip and create embeddings for labels

In [1]:
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms(
    "ViT-B-32", 
    pretrained="laion2b_s34b_b79k"
)

tokenizer = open_clip.get_tokenizer("ViT-B-32")

labels = [
    "an airplane",
    "a bird",
    "a car",
    "a cat",
    "a deer",
    "a dog",
    "a horse",
    "a monkey",
    "a ship",
    "a truck"
]

text = tokenizer(labels)
text_embeddings = model.encode_text(text)

/Users/dcmac14/Library/Caches/pypoetry/virtualenvs/deeplearning-4xETOx6U-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(text_embeddings)

10

In [3]:
import torch.nn.functional as F

def embeddings_to_class_probs(vision_embeddings, text_embeddings):
    vision_embeddings = vision_embeddings / vision_embeddings.norm(dim=-1, keepdim=True)
    text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)
    logits = vision_embeddings @ text_embeddings.T
    class_probs = F.softmax(100. * logits, dim=-1)
    return class_probs

### Get predictions on the training and testing dataset for open clip

In [4]:
import tqdm
import torch
from torchvision.datasets import STL10

dataset = STL10(
    root="./stl10",
    download=True,
    split="test"
)

num_correct = 0

for image, label in tqdm.tqdm(dataset):
    input_tensor = preprocess(image).unsqueeze(0)
    vision_embeddings = model.encode_image(input_tensor)
    output_class_probs = embeddings_to_class_probs(vision_embeddings, text_embeddings)
    output_label = torch.argmax(output_class_probs, dim=-1)
    num_correct += int(torch.count_nonzero(output_label == label))

accuracy = 100. * num_correct / len(dataset)

100%|██████████| 8000/8000 [04:51<00:00, 27.45it/s]


In [5]:
accuracy

96.675

### Training resnet18 from scratch on the STL10 dataset with no distillation


In [ ]:
from stl10_utils import (
    precompute_clip_stl10_train_image_embeddings,
    precompute_clip_stl10_test_image_embeddings,
    precompute_clip_stl10_text_embeddings,
    train_resnet18_from_scratch,
    train_resnet18_linear_probe_train_only
)

precompute_clip_stl10_train_image_embeddings()
precompute_clip_stl10_test_image_embeddings()
precompute_clip_stl10_text_embeddings()
train_resnet18_from_scratch()


/Users/dcmac14/Library/Caches/pypoetry/virtualenvs/deeplearning-4xETOx6U-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
5000it [00:00, 8787.94it/s]
8000it [00:00, 9257.65it/s]


Using device: mps for train_model_from_scratch


/Users/dcmac14/Library/Caches/pypoetry/virtualenvs/deeplearning-4xETOx6U-py3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1 - Test Loss: 1.7259, Test Accuracy: 34.29%
Epoch 2 - Test Loss: 1.6783, Test Accuracy: 39.52%
Epoch 3 - Test Loss: 1.4484, Test Accuracy: 46.79%
Epoch 4 - Test Loss: 1.3064, Test Accuracy: 51.29%
Epoch 5 - Test Loss: 1.2840, Test Accuracy: 52.86%
Epoch 6 - Test Loss: 1.2543, Test Accuracy: 53.89%
Epoch 7 - Test Loss: 1.3261, Test Accuracy: 55.14%
Epoch 8 - Test Loss: 1.2820, Test Accuracy: 57.17%
Epoch 9 - Test Loss: 1.3840, Test Accuracy: 56.45%
Epoch 10 - Test Loss: 1.3339, Test Accuracy: 57.70%
Finished Training


### Train linear probe version of the open clip transformer model first then use as the teacher for the resnet18 model. Only use the 5000 labeled training images.

In [ ]:
from stl10_utils import (
    precompute_clip_stl10_train_image_embeddings,
    precompute_clip_stl10_test_image_embeddings,
    precompute_clip_stl10_text_embeddings,
    train_resnet18_from_scratch,
    train_resnet18_linear_probe_train_only
)
train_resnet18_linear_probe_train_only()

/Users/dcmac14/Library/Caches/pypoetry/virtualenvs/deeplearning-nKFECO7W-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: mps for train_probe_model


100%|██████████| 125/125 [00:41<00:00,  3.02it/s] 

| EPOCH 0 | TRAIN LOSS 1.6945151030262815 | TEST ACC 96.525 |
Saving checkpoint to data/experiments/train_probe_model_linear/checkpoint_0.pth



100%|██████████| 125/125 [00:41<00:00,  3.01it/s] 

...

| EPOCH 47 | TRAIN LOSS 0.0019103355817732554 | TEST ACC 61.725 |
Saving checkpoint to data/experiments/train_resnet18_linear_probe_train_only/checkpoint_47.pth



100%|██████████| 125/125 [00:43<00:00,  2.87it/s]

| EPOCH 48 | TRAIN LOSS 0.0019364761884170998 | TEST ACC 61.625 |



100%|██████████| 125/125 [00:43<00:00,  2.87it/s]

| EPOCH 49 | TRAIN LOSS 0.0019169185602377298 | TEST ACC 61.875 |
Saving checkpoint to data/experiments/train_resnet18_linear_probe_train_only/checkpoint_49.pth


This results in a meager 62% accuracy, we can do better

### Train the resnet18 as the studuent with open_clip's linear probe version of the transformer as the teacher. Use the 5000 labeled images plus the 100000 unlabeled images for the training dataset.

In [ ]:
from stl10_utils import (
    precompute_clip_stl10_train_image_embeddings,
    precompute_clip_stl10_unlabeled_image_embeddings,
    precompute_clip_stl10_test_image_embeddings,
    precompute_clip_stl10_text_embeddings,
    train_resnet18_linear_probe
)

precompute_clip_stl10_train_image_embeddings()
precompute_clip_stl10_unlabeled_image_embeddings()
precompute_clip_stl10_test_image_embeddings()
precompute_clip_stl10_text_embeddings()


5000it [00:00, 6755.83it/s]
100000it [1:03:37, 26.20it/s]
8000it [00:01, 7360.21it/s]


Using device: mps for train_student_classification_model


In [2]:
from stl10_utils import (
    train_resnet18_linear_probe
)

train_resnet18_linear_probe()

/Users/dcmac14/Library/Caches/pypoetry/virtualenvs/deeplearning-nKFECO7W-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: mps for train_student_classification_model


  0%|          | 0/1641 [00:00<?, ?it/s]/Users/dcmac14/Library/Caches/pypoetry/virtualenvs/deeplearning-nKFECO7W-py3.10/lib/python3.10/site-packages/torch/nn/functional.py:3384: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
100%|██████████| 125/125 [04:09<00:00,  2.00s/it]

| EPOCH 0 | TRAIN LOSS 0.08073753968226263 | TEST ACC 62.725 |
Saving checkpoint to data/experiments/train_resnet18_linear_probe/checkpoint_0.pth



100%|██████████| 125/125 [00:43<00:00,  2.86it/s]

...

| EPOCH 18 | TRAIN LOSS 0.005285361894229876 | TEST ACC 96.025 |



100%|██████████| 125/125 [00:43<00:00,  2.90it/s]

| EPOCH 19 | TRAIN LOSS 0.004969530341758561 | TEST ACC 96.412 |



  5%|▌         | 83/1641 [00:09<02:57,  8.76it/s]


: 

#### This quickly returned much better results and is nearly at the same accuracy level to the base open clip transformer model.